In [12]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/AUNP_model

Mounted at /content/drive
/content/drive/MyDrive/AUNP_model


In [4]:
!pip install pillow_heif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 34.8 MB/s eta 0:00:00


In [5]:
from PIL import Image
from pillow_heif import register_heif_opener
import numpy as np
import os

In [ ]:
NEW = False # Generates new data label

In [13]:
PATH = "dataset"

file_dir = os.listdir(PATH)
file_dir.sort()

register_heif_opener()
if NEW:
  f = open("auto_label.csv", "a")
  f.write('image_id,label\n')

for i in range(len(file_dir)):
    image = Image.open(PATH + '/' + file_dir[i])
    image = image.resize((252, 336), resample=Image.NEAREST)
    image = image.crop((0, 84, 504, 588))
    image.save("training_data/%05d.png" % i,"png")
    if NEW: f.write("%05d.png,0\n" % i)

if NEW: f.close()

In [ ]:
print(len(file_dir))

283


In [ ]:
import tensorflow as tf

class Converter(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float32)])
    def __call__(self, x):
        return x.resize((252, 336), resample=Image.NEAREST)

model = Converter()
# (ro run your model) result = Squared(5.0) # This prints "25.0"
# (to generate a SavedModel) tf.saved_model.save(model, "saved_model_tf_dir")
concrete_func = model.__call__.get_concrete_function()

# Convert the model.
# Notes that for the versions earlier than TensorFlow 2.7, the
# from_concrete_functions API is able to work when there is only the first
# argument given:
# > converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func],
                                                            model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

AttributeError: in user code:

    File "C:\Users\peggy\AppData\Local\Temp\ipykernel_16632\3679767846.py", line 6, in __call__  *
        return x.resize((252, 336), resample=Image.NEAREST)

    AttributeError: 'Tensor' object has no attribute 'resize'
